In [1]:
import json
import csv
import numpy as np
from pprint import pprint

In [2]:
filenames = [
    'Aof-t20 (1024Hz)',
    'Aof-t21 (1024Hz)',
    'Aof-t22 (1024Hz)',
    'Aof-t23 (1024Hz)',
    'Aof-t24 (1024Hz)',
    'Aof-t25 (1024Hz)',
    'Beam-t1 (1024Hz)',
    'Beam-t2 (1024Hz)',
    'Beam-t3 (1024Hz)',
    'Beam-t6 (1024Hz)',
    'Beam-t7 (1024Hz)',
    'Beam-t8 (1024Hz)',
#     'Donut-t1 (1024Hz)',
#     'Donut-t2 (1024Hz)',
#     'Donut-t3 (1024Hz)',
#     'Donut-t4 (1024Hz)',
#     'Donut-t5 (1024Hz)',
#     'Donut-t6 (1024Hz)',
#     'Donut-t7 (1024Hz)',
#     'Eye-t1 (1024Hz)',
    'Eye-t2 (1024Hz)',
    'Eye-t3 (1024Hz)',
    'Eye-t4 (1024Hz)',
    'Eye-t5 (1024Hz)',
    'Eye-t6 (1024Hz)',
    'Eye-t7 (1024Hz)',
    'Faai-t1 (1024Hz)',
    'Faai-t2 (1024Hz)',
    'Faai-t3 (1024Hz)',
    'Faai-t4 (1024Hz)',
    'Faai-t5 (1024Hz)',
    'Faai-t6 (1024Hz)',
    'Fluke-t20 (1024Hz)',
    'Fluke-t21 (1024Hz)',
    'Fluke-t22 (1024Hz)',
    'Fluke-t23 (1024Hz)',
    'Fluke-t24 (1024Hz)',
    'Fluke-t25 (1024Hz)',
    'Fong-t1 (1024Hz)',
    'Fong-t2 (1024Hz)',
    'Fong-t3 (1024Hz)',
    'Fong-t4 (1024Hz)',
    'Fong-t5 (1024Hz)',
    'Fong-t6 (1024Hz)',
    'Fong-t7 (1024Hz)',
    'Fong-t8 (1024Hz)',
    'Joke-t1 (1024Hz)',
    'Joke-t2 (1024Hz)',
    'Joke-t3 (1024Hz)',
    'Joke-t4 (1024Hz)',
    'Joke-t5 (1024Hz)',
    'Joke-t6 (1024Hz)',
    'Pod-t20 (1024Hz)',
    'Pod-t21 (1024Hz)',
    'Pod-t22 (1024Hz)',
    'Pod-t23 (1024Hz)',
    'Pod-t24 (1024Hz)',
    'Pod-t25 (1024Hz)',
    'Pod-t26 (1024Hz)',
    'Tau-t11 (1024Hz)',
    'Tau-t13 (1024Hz)',
    'Tau-t14 (1024Hz)',
    'Tau-t15 (1024Hz)',
    'Tau-t16 (1024Hz)',
    'Tau-t17 (1024Hz)',
    'Tau-t18 (1024Hz)',
    'Toey-t1 (1024Hz)',
    'Toey-t2 (1024Hz)',
    'Toey-t3 (1024Hz)',
    'Toey-t4 (1024Hz)',
    'Toey-t5 (1024Hz)',
    'Toey-t6 (1024Hz)',
    'Toey-t7 (1024Hz)',
    'Tong-t1 (1024Hz)',
    'Tong-t2 (1024Hz)',
    'Tong-t3 (1024Hz)',
    'Tong-t4 (1024Hz)',
    'Tong-t5 (1024Hz)',
    'Tong-t6 (1024Hz)',
#     'Torn-t1 (1024Hz)',
#     'Torn-t2 (1024Hz)',
#     'Torn-t3 (1024Hz)',
    'Torn-t4 (1024Hz)',
    'Torn-t5 (1024Hz)',
    'Torn-t6 (1024Hz)',
    'Torn-t7 (1024Hz)',
    'Torn-t8 (1024Hz)',
    'Torn-t9 (1024Hz)'
]

In [3]:
first_dir = './data/label/1/'
second_dir = './data/label/2/'
first_out_dir = './data/label/sync/1/'
second_out_dir = './data/label/sync/2/'

infiles_1 = [first_dir + s + '.json' for s in filenames]
infiles_2 = [second_dir + s + '_LABEL.json' for s in filenames]
outfiles_1 = [first_out_dir + s + '.json' for s in filenames]
outfiles_2 = [second_out_dir + s + '.json' for s in filenames]

In [4]:
def remove_single_segment(label_data_1, label_data_2):
    i = 0
    j = 0
    remove_index_1 = []
    remove_index_2 = []
    
    while i < label_data_1.shape[0] and j < label_data_2.shape[0]:
        if label_data_2[j][0] > label_data_1[i][0] - 2048 and label_data_2[j][0] < label_data_1[i][0] + 2048:
            i += 1
            j += 1
        else:
            if label_data_1[i][0] < label_data_2[j][0]:
                # remove i
                remove_index_1.append(i)
                i += 1
            else:
                # remove j
                remove_index_2.append(j)
                j += 1
    while i < label_data_1.shape[0]:
        remove_index_1.append(i)
        i += 1
    while j < label_data_2.shape[0]:
        remove_index_2.append(j)
        j += 1
    
    mask_1 = np.ones(label_data_1.shape[0], dtype = bool)
    mask_1[remove_index_1] = False
    
    mask_2 = np.ones(label_data_2.shape[0], dtype = bool)
    mask_2[remove_index_2] = False

    return label_data_1[mask_1], label_data_2[mask_2] 

In [5]:
for i in range(0, len(filenames)):
    print("Processing " + filenames[i])
    with open(infiles_1[i]) as fin_1, open(infiles_2[i]) as fin_2, open(outfiles_1[i], 'w') as fout_1, open(outfiles_2[i], 'w') as fout_2:
        label_data_1 = json.load(fin_1)
        label_data_2 = json.load(fin_2)

        # Convert to numpy array
        np_label_data_1 = np.array(label_data_1)
        np_label_data_2 = np.array(label_data_2)

        print("1: {0:d}, 2: {1:d}".format(len(np_label_data_1), len(np_label_data_2)))
        
        np_label_data_1, np_label_data_2 = remove_single_segment(np_label_data_1, np_label_data_2)
        
        print("1: {0:d}, 2: {1:d}".format(len(np_label_data_1), len(np_label_data_2)))

        if len(np_label_data_1) != len(np_label_data_2):
            print("{0}: Length not equal".format(filenames[i]))
            continue

        fout_1.write(json.dumps(np_label_data_1.tolist()))
        fout_2.write(json.dumps(np_label_data_2.tolist()))

        fin_1.close()
        fin_2.close()
        fout_1.close()
        fout_2.close()
print("Done.")

Processing Aof-t20 (1024Hz)
1: 30, 2: 38
1: 30, 2: 30
Processing Aof-t21 (1024Hz)
1: 30, 2: 30
1: 30, 2: 30
Processing Aof-t22 (1024Hz)
1: 30, 2: 30
1: 30, 2: 30
Processing Aof-t23 (1024Hz)
1: 30, 2: 30
1: 30, 2: 30
Processing Aof-t24 (1024Hz)
1: 30, 2: 30
1: 30, 2: 30
Processing Aof-t25 (1024Hz)
1: 35, 2: 35
1: 35, 2: 35
Processing Beam-t1 (1024Hz)
1: 33, 2: 33
1: 33, 2: 33
Processing Beam-t2 (1024Hz)
1: 27, 2: 28
1: 27, 2: 27
Processing Beam-t3 (1024Hz)
1: 32, 2: 33
1: 32, 2: 32
Processing Beam-t6 (1024Hz)
1: 20, 2: 20
1: 20, 2: 20
Processing Beam-t7 (1024Hz)
1: 29, 2: 29
1: 29, 2: 29
Processing Beam-t8 (1024Hz)
1: 49, 2: 49
1: 48, 2: 48
Processing Eye-t2 (1024Hz)
1: 31, 2: 31
1: 31, 2: 31
Processing Eye-t3 (1024Hz)
1: 30, 2: 30
1: 30, 2: 30
Processing Eye-t4 (1024Hz)
1: 30, 2: 30
1: 30, 2: 30
Processing Eye-t5 (1024Hz)
1: 30, 2: 30
1: 30, 2: 30
Processing Eye-t6 (1024Hz)
1: 30, 2: 30
1: 30, 2: 30
Processing Eye-t7 (1024Hz)
1: 30, 2: 30
1: 30, 2: 30
Processing Faai-t1 (1024Hz)
1: 36,